In [1]:
# 说实话，用iris不是很好，因为一棵树基本上就很完美的分类了
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import numpy as np 
import pandas as pd 

In [2]:
iris = load_iris()
X = iris.data
y = iris.target
Xtrain,Xtest,ytrain,ytest = train_test_split(X,y,test_size=0.33, random_state=42)
Xtrain = StandardScaler().fit_transform(Xtrain)
Xtest = StandardScaler().fit_transform(Xtest)
print(Xtrain[:5])
ytrain

[[-0.13835603 -0.26550845  0.22229072  0.10894943]
 [ 2.14752625 -0.02631165  1.61160773  1.18499319]
 [-0.25866563 -0.02631165  0.39595535  0.37796037]
 [-0.8602136   1.16967238 -1.39857913 -1.37061074]
 [ 2.26783585 -0.50470526  1.66949594  1.05048772]]


array([1, 2, 1, 0, 2, 1, 0, 0, 0, 1, 2, 0, 0, 0, 1, 0, 1, 2, 0, 1, 2, 0,
       2, 2, 1, 1, 2, 1, 0, 1, 2, 0, 0, 1, 1, 0, 2, 0, 0, 1, 1, 2, 1, 2,
       2, 1, 0, 0, 2, 2, 0, 0, 0, 1, 2, 0, 2, 2, 0, 1, 1, 2, 1, 2, 0, 2,
       1, 2, 1, 1, 1, 0, 1, 1, 0, 1, 2, 2, 0, 1, 2, 2, 0, 2, 0, 1, 2, 2,
       1, 2, 1, 1, 2, 2, 0, 1, 2, 0, 1, 2])

In [3]:
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier(
    # boosting参数
    init=None,
    n_estimators=100,
    learning_rate=0.1, 
    subsample=0.8,
    loss='deviance',
    # 分割参数
    # max_features='sqrt',
    criterion='friedman_mse',
    # 分割停止参数
    min_samples_split =2, 
    min_impurity_split=None,
    min_impurity_decrease=0.0,
    max_depth=7,
    max_leaf_nodes=None,
    # 剪枝参数
    min_samples_leaf =1, 
    warm_start=False,
    random_state=10
)
gbc.fit(Xtrain,ytrain)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=7,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              n_iter_no_change=None, presort='auto', random_state=10,
              subsample=0.8, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [4]:
ypred = gbc.predict(Xtest)
ypred

array([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2,
       0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 0, 0, 2, 1, 1, 0,
       0, 1, 2, 2, 1, 2])

In [5]:
print(accuracy_score(ytest,ypred)) # 参数影响很大哦

1.0


In [13]:
# 网格搜索法
from sklearn.model_selection import GridSearchCV
gbc = GradientBoostingClassifier(
    # boosting参数
    init=None,
    # n_estimators=100,
    learning_rate=0.1, 
    subsample=0.8,
    loss='deviance',
    # 分割参数
    # max_features='sqrt',
    criterion='friedman_mse',
    # 分割停止参数
    min_samples_split =2, 
    min_impurity_split=None,
    min_impurity_decrease=0.0,
    max_depth=7,
    max_leaf_nodes=None,
    # 剪枝参数
    min_samples_leaf =1, 
    warm_start=False,
    random_state=10
)
n_estimator = np.arange(1,20,1)
print(n_estimator)
param_grid = {'n_estimators':n_estimator}
gsearch1 = GridSearchCV(estimator = gbc,param_grid=param_grid,iid=False,cv=5)
gsearch1.fit(Xtrain,ytrain)

# 计算测试集精度，利用最优的
ypred = gsearch1.predict(Xtest)
print(accuracy_score(ypred,ytest))

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
0.96


In [17]:
import pandas as pd

print(gsearch1.best_score_,gsearch1.best_params_)
params = gsearch1.cv_results_['params']
means = gsearch1.cv_results_['mean_test_score']
# print(params)
list = []
for param, mean in zip(params,means):
    param["mean_test_score"] = mean
    list.append(param)
pd.DataFrame(params)

0.9394736842105263 {'n_estimators': 1}


,n_estimators,mean_test_score
0,1,0.939474
1,2,0.939474
2,3,0.929950
3,4,0.929950
4,5,0.919950
5,6,0.919950
6,7,0.919950
7,8,0.909950
8,9,0.919950
9,10,0.919950
